In [2]:

# Stop warnings
# -------------
import warnings
warnings.filterwarnings("ignore")

# General imports
# ---------------
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

# MRI imports
# -----------
import nibabel as nb
import cortex

# Functions import
# ----------------
from utils import draw_cortex_vertex, set_pycortex_config_file

# GLM imports
# ----------------
from nilearn import image, datasets, plotting, surface
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.plotting import plot_design_matrix, plot_stat_map, plot_anat, plot_img

# Get inputs
# ----------
mount_dir = '/media/scratchms/data/PredictEye' #sys.argv[1]
subject = 'sub-01'#sys.argv[2]
session = 'ses-01'
task = 'SacLoc'
space = 'T1w'
run = 'run-1'
preproc = 'fmriprep_dct'

task_name = [task[:3], 'Fix' ]


tr_dur=1.2

file_img = "{cwd}/pp_data/{subject}/func/{subject}_task-{task}_space-{space}_{preproc}_avg.nii.gz".\
                format(cwd=mount_dir, subject=subject,task=task,space=space,preproc=preproc)

file_mask_img = '{cwd}/deriv_data/fmriprep/fmriprep/{subject}/{session}/func/{subject}_{session}_task-{task}_{run}_space-{space}_desc-brain_mask.nii.gz'.\
                format(cwd=mount_dir, subject=subject, session=session, task=task, run=run, space=space)


output_folder = '{cwd}/pp_data/{subject}/glm/fit/'.format(cwd=mount_dir, subject=subject)


try:
    os.makedirs(output_folder)
except:
    pass


# create design table
# design_file_run1 = '{cwd}/bids_data/{subject}/{session}/func/{subject}_{session}_task-{task}_{run}_events.tsv'.\
#                         format(cwd=mount_dir, subject=subject, session=session, task=task, run='run-01')
# events_run1 = pd.read_table(design_file_run1)

# design_file_run2 = '{cwd}/bids_data/{subject}/{session}/func/{subject}_{session}_task-{task}_{run}_events.tsv'.\
#                         format(cwd=mount_dir, subject=subject, session=session, task=task, run='run-02')
# events_run2 = pd.read_table(design_file_run2)
onset = [    0*tr_dur,
             16*tr_dur,
             (16+32)*tr_dur,
             (16+32+16)*tr_dur,
             (16+32+16+32)*tr_dur,
             (16+32+16+32+16)*tr_dur,
             (16+32+16+32+16+32)*tr_dur,
             (16+32+16+32+16+32+16)*tr_dur,
             (16+32+16+32+16+32+16+32)*tr_dur]

duration = [ 16*tr_dur,
             32*tr_dur,
             16*tr_dur,
             32*tr_dur,
             16*tr_dur,
             32*tr_dur,
             16*tr_dur,
             32*tr_dur,
             16*tr_dur,
             ]

trial_type = [task_name[0],task_name[1],task_name[0],task_name[1],
              task_name[0],task_name[1],task_name[0],task_name[1],task_name[0]]

data = {'onset':onset,
        'duration':duration,
        'trial_type':trial_type}

events_run1_glm = pd.DataFrame(data)


# first level GLM


fmri_img = file_img
output_fn = '{output_folder}{subject}_task-{task}_space-{space}_{preproc}_deriv.nii.gz'.\
            format(output_folder=output_folder, subject=subject,task=task,space=space,preproc=preproc)
mask_img = nb.load(file_mask_img)

fmri_glm = FirstLevelModel(t_r=tr_dur,
                        noise_model='ar1',
                        standardize=False,
                        hrf_model='spm',
                        drift_model=None,
                        mask_img=mask_img)

fmri_glm = fmri_glm.fit(fmri_img, events_run1_glm)

# design matrix
design_matrix = fmri_glm.design_matrices_[0]
#     plot_design_matrix(design_matrix)
#     plt.show()

# contrast
exec('conditions = { task_name[1]: np.array([1., 0., 0.]), task_name[0]: np.array([0., 1., 0.])}')

contrast = conditions[task_name[0]] - conditions[task_name[1]]
plotting.plot_contrast_matrix(contrast, design_matrix=design_matrix)

eff_map = fmri_glm.compute_contrast(sac_minus_fix,
                                    output_type='effect_size')

z_map = fmri_glm.compute_contrast(sac_minus_fix,
                                    output_type='z_score')



# stats maps
# fdr_map, threshold_fdr = threshold_stats_img(z_map, alpha=.05, height_control='fdr')
# fdr_cluster_map, threshold_fdr_cluster = threshold_stats_img(z_map, alpha=.05, height_control='fdr', cluster_threshold=30)



# Save results
img = nb.load(file_img)
deriv = np.zeros((img.shape[0],img.shape[1],img.shape[2],2))*np.nan
# deriv[...,0]  = fpr_map.dataobj
# deriv[...,1]  = bonferroni_map.dataobj
# deriv[...,2]  = fdr_map.dataobj
# deriv[...,3]  = fdr_cluster_map.dataobj

# deriv = deriv.astype(np.float32)
# new_img = nb.Nifti1Image(dataobj = deriv, affine = img.affine, header = img.header)
# new_img.to_filename(output_fn)

NameError: name 'task_name' is not defined

In [10]:
zscore = stats.norm.ppf(.05)
print(zscore)

-1.6448536269514729


In [15]:
cdf = stats.norm.cdf(zscore)
cdf

0.049999999999999975